In [1]:
import pandas as pd

df = pd.read_csv("./output_data/summary/summary_large.csv")

In [2]:
length_cols = ['plan_length_1', 'plan_length_2', 'plan_length_3']
plan_cols   = ['plan_1', 'plan_2', 'plan_3']

# Filter rows where any plan_length is less than 6
df_short = df[df[length_cols].lt(6).any(axis=1)]

# Flatten
records = []

for _, row in df_short.iterrows():
    for i in range(1, 4):
        if row[f'plan_length_{i}'] < 6:
            records.append({
                'plan': row[f'plan_{i}'],
                'start_name': row['start_name'],
                'destination_name': row['destination_name'],
                'start_coordinates': row['start_coordinates'],
                'destination_coordinates': row['destination_coordinates']
            })

flattened_df = pd.DataFrame(records)

# flattened_df.to_csv("short_plans.csv", index=False)


### Check for duplicate detailed plans

In [3]:
import ast


detailed_df = pd.read_csv("./output_data/detailed_travel_routes.csv")

detailed_df['Domain'] = detailed_df['Domain'].apply(lambda x: ast.literal_eval(x.replace('coordinates = ', '').strip()))

detailed_df['Plan'] = detailed_df['Plan'].apply(ast.literal_eval)

detailed_df['Detailed Plan'] = detailed_df['Detailed Plan'].apply(ast.literal_eval)

def encode_plans(detailed_plan):
    for idx in range(len(detailed_plan)):
        detailed_plan[idx] = str(detailed_plan[idx])
    return detailed_plan

detailed_df['Detailed Plan'] = detailed_df['Detailed Plan'].apply(encode_plans)

has_duplicates = detailed_df['Detailed Plan'].apply(lambda x: len(x) != len(set(x)))

# Rows with duplicate entries in Detailed_Plan
duplicates_df = detailed_df[has_duplicates]

print(len(duplicates_df))

0


### Check for duplicate plans

In [4]:
detailed_df['Plan'] = detailed_df['Plan'].apply(encode_plans)

has_duplicates_1 = detailed_df['Plan'].apply(lambda x: len(x) != len(set(x)))

# Rows with duplicate entries in Detailed_Plan
duplicates_df_1 = detailed_df[has_duplicates_1]

print(len(duplicates_df_1))

0


### Final Check

Check for instances where the plan is the same, but detailed plan is different.
(For debugging)

In [5]:
only_in_df1 = duplicates_df[~duplicates_df['id'].isin(duplicates_df_1['id'])]

# IDs in df2 but not in df1
only_in_df2 = duplicates_df_1[~duplicates_df_1['id'].isin(duplicates_df['id'])]

only_in_df2

,id,Domain,Goal,Plan,Detailed Plan
